In [1]:
import pandas as pd
import numpy as np
import torch
import os
import json
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

from autogluon.tabular import TabularPredictor
import lightgbm as lgb
import catboost as cb
import xgboost as xgb

class EnhancedShipAnomalyDetector:
    """향상된 선박 이상행동 탐지 시스템 - 완전 버전"""
    
    def __init__(self):
        self.rule_feature_names = []
        self.enhanced_feature_names = []
        self.scaler = StandardScaler()
        self.rule_weights = {}
        self.rule_stats = {}
        
    def save_rule_weights(self, weights_path='./rule_weights.json'):
        """규칙 가중치를 파일로 저장"""
        if hasattr(self, 'rule_weights'):
            with open(weights_path, 'w') as f:
                json.dump(self.rule_weights, f, indent=2)
            print(f"💾 규칙 가중치 저장: {weights_path}")
            
            # 통계 정보도 저장
            stats_path = weights_path.replace('.json', '_stats.json')
            if hasattr(self, 'rule_stats'):
                with open(stats_path, 'w') as f:
                    json.dump(self.rule_stats, f, indent=2)
                print(f"📊 규칙 통계 저장: {stats_path}")
        else:
            print("⚠️ 저장할 가중치가 없습니다")

    def load_rule_weights(self, weights_path='./rule_weights.json'):
        """저장된 규칙 가중치를 로드"""
        if os.path.exists(weights_path):
            with open(weights_path, 'r') as f:
                self.rule_weights = json.load(f)
            print(f"📂 규칙 가중치 로드: {weights_path}")
            print("🔧 로드된 가중치:")
            for rule, weight in self.rule_weights.items():
                print(f"  {rule}: {weight:.3f}")
            
            # 통계 정보도 로드 (있다면)
            stats_path = weights_path.replace('.json', '_stats.json')
            if os.path.exists(stats_path):
                with open(stats_path, 'r') as f:
                    self.rule_stats = json.load(f)
                print(f"📊 규칙 통계도 로드됨")
            
            return True
        else:
            print(f"⚠️ 가중치 파일을 찾을 수 없습니다: {weights_path}")
            print("기본 가중치를 사용합니다")
            return False

    def calculate_rule_weights(self, df):
        """훈련 데이터에서 각 규칙의 실제 정확도 계산"""
        print("📊 실제 데이터에서 규칙 가중치 계산...")
        
        rule_weights = {}
        rule_stats = {}
        
        rule_columns = [
            'rule1_foreign_slow_non_anchor', 'rule2_ais_off_non_anchor',
            'rule3_special_zone_frequent', 'rule4_sharp_turn_non_anchor', 
            'rule5_slow_back_forth', 'rule6_restricted_zone'
        ]
        
        for rule in rule_columns:
            if rule in df.columns:
                # 해당 규칙에 적용되는 케이스들
                rule_cases = df[df[rule] == 1]
                
                if len(rule_cases) > 0:
                    # 실제 이상 비율 계산
                    accuracy = rule_cases['result'].mean()
                    rule_weights[rule] = accuracy
                    rule_stats[rule] = {
                        'total_cases': len(rule_cases),
                        'anomaly_cases': int(rule_cases['result'].sum()),
                        'accuracy': accuracy
                    }
                    print(f"  {rule}: {len(rule_cases)}건 중 {int(rule_cases['result'].sum())}건 이상 → 정확도 {accuracy:.3f}")
                else:
                    rule_weights[rule] = 0.0
                    rule_stats[rule] = {'total_cases': 0, 'anomaly_cases': 0, 'accuracy': 0.0}
                    print(f"  {rule}: 해당 케이스 없음 → 가중치 0.0")
            else:
                rule_weights[rule] = 0.0
                rule_stats[rule] = {'total_cases': 0, 'anomaly_cases': 0, 'accuracy': 0.0}
        
        self.rule_weights = rule_weights
        self.rule_stats = rule_stats
        
        print(f"✅ 규칙 가중치 계산 완료")
        return rule_weights
    
    def create_rule_based_features(self, df, is_training=False, weights_path='./rule_weights.json'):
        """6가지 규칙 기반 특성 생성"""
        print("🔧 도메인 지식 기반 특성 엔지니어링...")
        
        df_enhanced = df.copy()
        
        # 규칙 1: 외국국적 + 저속항해 + 비정박지
        df_enhanced['rule1_foreign_slow_non_anchor'] = (
            (df['is_korean_ship'] == 0) & 
            (df['anchor_zone_ratio'] < 0.8) & 
            ((df['low_speed_ratio'] > 0.7) | (df['avg_sog'] <= 5))
        ).astype(int)
        
        # 규칙 2: AIS OFF 이상 + 비정박지
        df_enhanced['rule2_ais_off_non_anchor'] = (
            (df['anchor_zone_ratio'] < 0.8) & 
            (df['off_events'] >= 3)
        ).astype(int)
        
        # 규칙 3: 특수해역 다중진입
        df_enhanced['rule3_special_zone_frequent'] = (
            df['num_zone_entries'] > 50
        ).astype(int)
        
        # 규칙 4: 급변침 + 비정박지
        df_enhanced['rule4_sharp_turn_non_anchor'] = (
            (df['anchor_zone_ratio'] < 0.8) & 
            (df['sharp_turns'] >= 1)
        ).astype(int)
        
        # 규칙 5: 저속 + 왕복항해
        df_enhanced['rule5_slow_back_forth'] = (
            (df['avg_sog'] <= 5) & 
            (df['back_forth_count'] >= 2)
        ).astype(int)
        
        # 규칙 6: 금지구역 진입
        df_enhanced['rule6_restricted_zone'] = df['restricted_zone_flag']
        
        # 가중치 처리
        if is_training and 'result' in df.columns:
            # 훈련 시: 실제 데이터에서 가중치 계산
            rule_weights = self.calculate_rule_weights(df_enhanced)
            # 계산한 가중치를 파일로 저장
            self.save_rule_weights(weights_path)
        else:
            # 테스트 시: 저장된 가중치 로드
            loaded = self.load_rule_weights(weights_path)
            if loaded:
                rule_weights = self.rule_weights
            else:
                # 로드 실패 시 기본값 사용
                rule_weights = {
                    'rule6_restricted_zone': 1.0,
                    'rule1_foreign_slow_non_anchor': 0.97,
                    'rule3_special_zone_frequent': 0.92,
                    'rule4_sharp_turn_non_anchor': 0.72,
                    'rule2_ais_off_non_anchor': 0.0,
                    'rule5_slow_back_forth': 0.0
                }
                self.rule_weights = rule_weights
        
        # 종합 규칙 점수 (실제 데이터 기반 가중합)
        df_enhanced['rule_composite_score'] = 0
        for rule, weight in rule_weights.items():
            if rule in df_enhanced.columns:
                df_enhanced['rule_composite_score'] += df_enhanced[rule] * weight
        
        # 규칙 기반 예측
        df_enhanced['rule_based_prediction'] = (
            (df_enhanced['rule1_foreign_slow_non_anchor'] == 1) |
            (df_enhanced['rule2_ais_off_non_anchor'] == 1) |
            (df_enhanced['rule3_special_zone_frequent'] == 1) |
            (df_enhanced['rule4_sharp_turn_non_anchor'] == 1) |
            (df_enhanced['rule5_slow_back_forth'] == 1) |
            (df_enhanced['rule6_restricted_zone'] == 1)
        ).astype(int)
        
        rule_features = [
            'rule1_foreign_slow_non_anchor', 'rule2_ais_off_non_anchor',
            'rule3_special_zone_frequent', 'rule4_sharp_turn_non_anchor',
            'rule5_slow_back_forth', 'rule6_restricted_zone',
            'rule_composite_score', 'rule_based_prediction'
        ]
        
        self.rule_feature_names = rule_features
        print(f"✅ 규칙 기반 특성 {len(rule_features)}개 생성 완료")
        
        return df_enhanced
    
    
    def get_enhanced_hyperparameters(self, gpu_available=False):
        """향상된 하이퍼파라미터 (불균형 데이터 고려)"""
        
        cpu_count = os.cpu_count() or 2
        
        if gpu_available:
            hyperparameters = {
                # XGBoost with class balancing
                'XGB': [
                    {
                        'n_estimators': 500,
                        'learning_rate': 0.05,
                        'max_depth': 8,
                        'tree_method': 'gpu_hist',
                        'gpu_id': 0,
                        'objective': 'binary:logistic',
                        'eval_metric': 'auc',
                        'scale_pos_weight': 0.62,  # 불균형 조정
                        'subsample': 0.8,
                        'colsample_bytree': 0.8
                    }
                ],
                
                # LightGBM with focal loss
                'GBM': [
                    {
                        'num_boost_round': 500,
                        'learning_rate': 0.05,
                        'device': 'gpu',
                        'objective': 'binary',
                        'metric': 'auc',
                        'is_unbalance': True,
                        'feature_fraction': 0.8,
                        'bagging_fraction': 0.8,
                        'bagging_freq': 5
                    }
                ],
                
                # CatBoost with auto class balancing
                'CAT': [
                    {
                        'iterations': 500,
                        'learning_rate': 0.05,
                        'depth': 8,
                        'task_type': 'GPU',
                        'devices': '0',
                        'auto_class_weights': 'Balanced',
                        'eval_metric': 'AUC',
                        'bootstrap_type': 'Bernoulli',
                        'subsample': 0.8
                    }
                ],
                
                # Neural Network
                'NN_TORCH': [
                    {
                        'num_epochs': 200,
                        'learning_rate': 0.001,
                        'batch_size': 512,
                        'activation': 'relu',
                        'dropout_prob': 0.3,
                        'weight_decay': 1e-4
                    }
                ]
            }
        else:
            # CPU 최적화 버전
            hyperparameters = {
                'GBM': {
                    'num_boost_round': 300,
                    'learning_rate': 0.1,
                    'is_unbalance': True,
                    'metric': 'auc'
                },
                'CAT': {
                    'iterations': 300,
                    'learning_rate': 0.1,
                    'auto_class_weights': 'Balanced'
                },
                'XGB': {
                    'n_estimators': 300,
                    'learning_rate': 0.1,
                    'scale_pos_weight': 0.62,
                    'eval_metric': 'auc'
                },
                'RF': {
                    'n_estimators': 200,
                    'max_depth': 15,
                    'class_weight': 'balanced',
                    'n_jobs': cpu_count
                }
            }
        
        return hyperparameters
    
    def train_enhanced_model(self, file_path, time_limit=1800, quality='high_quality'):
        """향상된 모델 훈련"""
        print("🌟 향상된 선박 이상행동 탐지 모델 훈련")
        print("=" * 60)
        
        # GPU 상태 확인
        gpu_available = torch.cuda.is_available()
        print(f"🔍 GPU 사용 가능: {gpu_available}")
        
        # 데이터 로드
        print("📁 데이터 로딩...")
        df = pd.read_csv(file_path)
        print(f"✅ 원본 데이터: {df.shape}")
        
        # 타겟 전처리
        if 'result' in df.columns:
            if df['result'].dtype == 'object':
                df['result'] = df['result'].map({'True': 1, 'False': 0})
            elif df['result'].dtype == 'bool':
                df['result'] = df['result'].astype(int)
        
        # 특성 엔지니어링 (훈련 모드)
        df_final = self.create_rule_based_features(df, is_training=True)
        #df_final = self.create_advanced_features(df_enhanced)
        
        print(f"📊 최종 데이터: {df_final.shape}")
        
        # 타겟 분포 확인
        target_dist = df_final['result'].value_counts()
        print(f"📈 타겟 분포: {dict(target_dist)}")
        print(f"📊 이상선박 비율: {target_dist[1]/len(df_final):.2%}")
        
        # 특성과 타겟 분리
        feature_columns = [col for col in df_final.columns if col not in ['result', 'MMSI']]
        X = df_final[feature_columns]
        y = df_final['result']
        
        print(f"🔧 사용할 특성: {len(feature_columns)}개")
        
        # 데이터 분할 (층화 추출)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        print(f"📊 훈련 데이터: {X_train.shape[0]:,}개")
        print(f"📊 테스트 데이터: {X_test.shape[0]:,}개")
        
        # 훈련 데이터 준비
        train_data = X_train.copy()
        train_data['target'] = y_train
        
        # 모델 경로 설정
        model_path = './enhanced_ship_anomaly_models_b3'
        if os.path.exists(model_path):
            import shutil
            shutil.rmtree(model_path)
        
        # AutoGluon 설정
        predictor = TabularPredictor(
            label='target',
            problem_type='binary',
            eval_metric='roc_auc',  # AUC로 변경 (불균형 데이터에 적합)
            path=model_path,
            verbosity=2
        )
        
        # 하이퍼파라미터 설정
        hyperparameters = self.get_enhanced_hyperparameters(gpu_available)
        
        # AG args 설정
        ag_args_fit = {
            'num_gpus': 1 if gpu_available else 0,
            'num_cpus': os.cpu_count() or 2,
            'auto_stack': True
        }
        
        print(f"\n⏳ 모델 훈련 시작... (제한시간: {time_limit//60}분)")
        start_time = pd.Timestamp.now()
        
        try:
            predictor.fit(
                train_data,
                time_limit=time_limit,
                presets=quality,
                hyperparameters=hyperparameters,
                ag_args_fit=ag_args_fit,
                holdout_frac=0.15,  # 검증용
                num_bag_folds=5,
                num_bag_sets=1,
                num_stack_levels=2,  # 스태킹 활용
                verbosity=2,
                dynamic_stacking=True,  # 동적 스태킹
            )
            
            end_time = pd.Timestamp.now()
            training_time = (end_time - start_time).total_seconds()
            print(f"✅ 훈련 완료! (소요시간: {training_time/60:.1f}분)")
            
        except Exception as e:
            print(f"⚠️ 훈련 중 오류: {e}")
            return None, None, None, None
        
        # GPU 메모리 정리
        if gpu_available:
            torch.cuda.empty_cache()
        
        return predictor, X_test, y_test, training_time
    
    def comprehensive_evaluation(self, predictor, X_test, y_test, training_time):
        """포괄적 모델 평가"""
        print(f"\n📈 포괄적 성능 평가")
        print("=" * 40)
        
        # 예측 수행
        start_time = pd.Timestamp.now()
        y_pred = predictor.predict(X_test)
        y_pred_proba = predictor.predict_proba(X_test)
        end_time = pd.Timestamp.now()
        
        prediction_time = (end_time - start_time).total_seconds()
        
        # 기본 성능 지표
        accuracy = accuracy_score(y_test, y_pred)
        auc_score = roc_auc_score(y_test, y_pred_proba.iloc[:, 1])
        
        print(f"🎯 핵심 성능 지표:")
        print(f"  정확도: {accuracy:.4f}")
        print(f"  AUC 점수: {auc_score:.4f}")
        print(f"  훈련 시간: {training_time/60:.1f}분")
        print(f"  예측 시간: {prediction_time:.2f}초")
        print(f"  예측 속도: {len(X_test)/prediction_time:.0f} 샘플/초")
        
        # 상세 분류 리포트
        print(f"\n📊 상세 분류 리포트:")
        print(classification_report(y_test, y_pred, target_names=['정상', '이상']))
        
        # 혼동 행렬
        cm = confusion_matrix(y_test, y_pred)
        print(f"\n🔍 혼동 행렬:")
        print(f"실제\\예측    정상    이상")
        print(f"정상        {cm[0,0]:4d}   {cm[0,1]:4d}")
        print(f"이상        {cm[1,0]:4d}   {cm[1,1]:4d}")
        
        # 모델 리더보드
        try:
            leaderboard = predictor.leaderboard(silent=True)
            print(f"\n🏆 모델 리더보드 (상위 5개):")
            top_models = leaderboard.head(5)
            for idx, row in top_models.iterrows():
                model_name = row['model']
                score = row['score_val']
                print(f"  {model_name}: {score:.4f}")
        except:
            leaderboard = pd.DataFrame()
        
        # 규칙 기반 vs ML 성능 비교
        if 'rule_based_prediction' in X_test.columns:
            rule_accuracy = accuracy_score(y_test, X_test['rule_based_prediction'])
            print(f"\n⚖️ 규칙 기반 vs ML 비교:")
            print(f"  규칙 기반 정확도: {rule_accuracy:.4f}")
            print(f"  ML 모델 정확도: {accuracy:.4f}")
            print(f"  개선율: {((accuracy - rule_accuracy) / rule_accuracy * 100):+.1f}%")
        
        # 특성 중요도 (가능한 경우)
        try:
            feature_importance = predictor.feature_importance(X_test)
            print(f"\n🔍 상위 10개 중요 특성:")
            top_features = feature_importance.head(10)
            for feature, importance in top_features.items():
                print(f"  {feature}: {importance:.4f}")
        except:
            print(f"\n⚠️ 특성 중요도 분석 불가")
        
        return {
            'accuracy': accuracy,
            'auc': auc_score,
            'training_time': training_time,
            'prediction_time': prediction_time,
            'leaderboard': leaderboard
        }
    
    def hybrid_prediction(self, predictor, X_new):
        """하이브리드 예측 (규칙 + ML)"""
        # ML 예측
        ml_pred = predictor.predict(X_new)
        ml_proba = predictor.predict_proba(X_new)
        
        # 규칙 기반 예측
        if 'rule_based_prediction' in X_new.columns:
            rule_pred = X_new['rule_based_prediction'].values
            
            # 하이브리드 예측: 규칙이 이상이면 이상, 아니면 ML 예측 사용
            hybrid_pred = np.maximum(rule_pred, ml_pred)
            
            return hybrid_pred, ml_proba
        else:
            return ml_pred, ml_proba

def train_ship_anomaly_model(file_path, time_limit=7200, quality='high_quality'):
    """모델 훈련 파이프라인"""
    
    detector = EnhancedShipAnomalyDetector()
    
    try:
        # 모델 훈련
        predictor, X_test, y_test, training_time = detector.train_enhanced_model(
            file_path=file_path,
            time_limit=time_limit,
            quality=quality
        )
        
        if predictor is None:
            print("❌ 훈련 실패")
            return None, None, None
        
        # 성능 평가
        results = detector.comprehensive_evaluation(
            predictor, X_test, y_test, training_time
        )
        
        print("\n" + "=" * 60)
        print("🎉 훈련 파이프라인 완료!")
        print(f"🎯 최종 AUC: {results['auc']:.4f}")
        print(f"🎯 최종 정확도: {results['accuracy']:.4f}")
        print(f"⏱️ 총 훈련 시간: {training_time/60:.1f}분")
        print("=" * 60)
        
        return detector, predictor, results
        
    except Exception as e:
        print(f"❗ 훈련 파이프라인 오류: {e}")
        return None, None, None

def predict_test_data(test_file_path, model_path='./enhanced_ship_anomaly_models_b3', output_path='./test_predictions_b3.csv'):
    """학습된 모델로 test 데이터 예측"""
    
    print("🔮 Test 데이터 예측 시작")
    print("=" * 50)
    
    # 디텍터 인스턴스 생성
    detector = EnhancedShipAnomalyDetector()
    
    try:
        # 학습된 모델 로드
        print(f"📂 모델 로딩: {model_path}")
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"모델 경로가 존재하지 않습니다: {model_path}")
        
        predictor = TabularPredictor.load(model_path)
        print("✅ 모델 로딩 완료")
        
        # 테스트 데이터 로드
        print(f"📁 테스트 데이터 로딩: {test_file_path}")
        test_df = pd.read_csv(test_file_path)
        print(f"✅ 테스트 데이터 로딩 완료: {test_df.shape}")
        
        # MMSI 보존
        mmsi_column = test_df['MMSI'].copy() if 'MMSI' in test_df.columns else None
        
        # 동일한 특성 엔지니어링 적용 (테스트 모드)
        print("⚙️ 특성 엔지니어링 적용...")
        test_final = detector.create_rule_based_features(test_df, is_training=False)
        #test_final = detector.create_advanced_features(test_enhanced)
        
        # 예측용 특성 준비
        feature_columns = [col for col in test_final.columns if col not in ['result', 'MMSI']]
        X_test = test_final[feature_columns]
        
        print(f"🔧 예측용 특성: {len(feature_columns)}개")
        print(f"📊 예측할 데이터: {X_test.shape[0]:,}개")
        
        # 하이브리드 예측 수행
        print("🔮 예측 수행...")
        start_time = pd.Timestamp.now()
        
        hybrid_pred, ml_proba = detector.hybrid_prediction(predictor, X_test)
        
        end_time = pd.Timestamp.now()
        prediction_time = (end_time - start_time).total_seconds()
        
        print(f"✅ 예측 완료! (소요시간: {prediction_time:.2f}초)")
        print(f"🚀 예측 속도: {len(X_test)/prediction_time:.0f} 샘플/초")
        
        # 결과를 TRUE/FALSE로 변환
        result_labels = ['False' if pred == 0 else 'True' for pred in hybrid_pred]
        
        # 결과 DataFrame 생성
        result_df = pd.DataFrame()
        
        if mmsi_column is not None:
            result_df['MMSI'] = mmsi_column
        
        result_df['result'] = result_labels
        
        # 예측 결과 통계
        pred_counts = pd.Series(result_labels).value_counts()
        print(f"\n📊 예측 결과 분포:")
        for label, count in pred_counts.items():
            percentage = count / len(result_labels) * 100
            print(f"  {label}: {count:,}개 ({percentage:.1f}%)")
        
        # 규칙 기반 vs 하이브리드 비교
        if 'rule_based_prediction' in X_test.columns:
            rule_anomalies = X_test['rule_based_prediction'].sum()
            hybrid_anomalies = sum(hybrid_pred)
            print(f"\n⚖️ 규칙 vs 하이브리드:")
            print(f"  규칙 기반 이상: {rule_anomalies:,}개")
            print(f"  하이브리드 이상: {hybrid_anomalies:,}개")
            print(f"  ML 추가 탐지: {hybrid_anomalies - rule_anomalies:,}개")
        
        # 결과 저장
        result_df.to_csv(output_path, index=False)
        print(f"\n💾 결과 저장 완료: {output_path}")
        
        # 샘플 결과 출력
        print(f"\n📋 예측 결과 샘플 (상위 10개):")
        print(result_df.head(10).to_string(index=False))
        
        # 고신뢰도 이상 선박
        high_confidence_anomalies = result_df[
            (result_df['result'] == 'True') & 
            (result_df['confidence'] > 0.8)
        ]
        
        if len(high_confidence_anomalies) > 0:
            print(f"\n🚨 고신뢰도 이상 선박 ({len(high_confidence_anomalies)}개):")
            print(high_confidence_anomalies.head().to_string(index=False))
        
        print(f"\n🎉 예측 완료!")
        print(f"📁 결과 파일: {output_path}")
        print("=" * 50)
        
        return result_df
        
    except Exception as e:
        print(f"❌ 예측 중 오류 발생: {e}")
        return None

# 사용 예시
if __name__ == "__main__":
    print("🚀 완전한 선박 이상행동 탐지 시스템")
    print("훈련 → 평가 → 테스트 예측 파이프라인")
    print("=" * 60)
    
    # 1. 모델 훈련
    print("\n🔥 STEP 1: 모델 훈련")
    print("-" * 30)
    
    train_file_path = './train/merged_output.csv'  # 훈련 데이터 경로
    
    detector, predictor, results = train_ship_anomaly_model(
        file_path=train_file_path,
        time_limit=7200,  # 30분
        quality='high_quality'
    )
    
    if detector and predictor:
        print(f"\n✅ 훈련 성공!")
        print(f"💾 모델 저장 경로: ./enhanced_ship_anomaly_models_b2")
        print(f"💾 규칙 가중치 저장: ./rule_weights.json")
        
        # 2. 테스트 예측
        print("\n🔮 STEP 2: 테스트 데이터 예측")
        print("-" * 30)
        
        test_file_path = './20240701.csv'  # 테스트 데이터 경로
        output_path = './test_predictions_basic_2.csv'  # 결과 저장 경로
        
        predictions = predict_test_data(
            test_file_path=test_file_path,
            model_path='./enhanced_ship_anomaly_models_b2',
            output_path=output_path
        )
        
        if predictions is not None:
            print(f"\n🎉 전체 파이프라인 성공!")
            print(f"📊 총 {len(predictions):,}개 선박 예측 완료")
            print(f"💾 최종 결과: {output_path}")
        else:
            print("❌ 테스트 예측 실패")
            
    else:
        print("❌ 훈련 실패")

🚀 완전한 선박 이상행동 탐지 시스템
훈련 → 평가 → 테스트 예측 파이프라인

🔥 STEP 1: 모델 훈련
------------------------------
🌟 향상된 선박 이상행동 탐지 모델 훈련
🔍 GPU 사용 가능: True
📁 데이터 로딩...
✅ 원본 데이터: (2678129, 27)
🔧 도메인 지식 기반 특성 엔지니어링...
📊 실제 데이터에서 규칙 가중치 계산...
  rule1_foreign_slow_non_anchor: 217726건 중 213711건 이상 → 정확도 0.982
  rule2_ais_off_non_anchor: 해당 케이스 없음 → 가중치 0.0
  rule3_special_zone_frequent: 137666건 중 130756건 이상 → 정확도 0.950
  rule4_sharp_turn_non_anchor: 55536건 중 39009건 이상 → 정확도 0.702
  rule5_slow_back_forth: 해당 케이스 없음 → 가중치 0.0
  rule6_restricted_zone: 584007건 중 584007건 이상 → 정확도 1.000
✅ 규칙 가중치 계산 완료
💾 규칙 가중치 저장: ./rule_weights.json
📊 규칙 통계 저장: ./rule_weights_stats.json
✅ 규칙 기반 특성 8개 생성 완료
📊 최종 데이터: (2678129, 35)
📈 타겟 분포: {1: 1633217, 0: 1044912}
📊 이상선박 비율: 60.98%
🔧 사용할 특성: 33개
📊 훈련 데이터: 2,142,503개
📊 테스트 데이터: 535,626개


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.12.7
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #107-Ubuntu SMP Wed Feb 7 13:26:48 UTC 2024
CPU Count:          2
Memory Avail:       15.19 GB / 24.00 GB (63.3%)
Disk Space Avail:   449.89 GB / 511.75 GB (87.9%)
Presets specified: ['high_quality']
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=5, num_bag_sets=1
Note: `save_bag_folds=False`! This will greatly reduce peak disk usage during fit (by ~5x), but runs the risk of an out-of-memory error during model refit if memory is small relative to the data size.
	You can avoid this risk by setting `save_bag_folds=True`.
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels`


⏳ 모델 훈련 시작... (제한시간: 120분)


2025-05-29 02:58:28,661	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.8.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
	Running DyStack sub-fit in a ray process to avoid memory leakage. Enabling ray logging (enable_ray_logging=True). Specify `ds_args={'enable_ray_logging': False}` if you experience logging issues.
2025-05-29 02:58:33,053	INFO worker.py:1843 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 
		Context path: "/home/elicer/ai/enhanced_ship_anomaly_models_b3/ds_sub_fit/sub_fit_ho"
(_dystack pid=4124654) Running DyStack sub-fit ...
(_dystack pid=4124654) Beginning AutoGluon training ... Time limit = 1793s
(_dystack pid=4124654) AutoGluon will save models to "/home/elicer/ai/enhanced_ship_anomaly_models_b3/ds_sub_fit/sub_fit_ho"
(_dystack pid=4124654) Train Data Rows:    1904447
(_dystack pid=4124654) Train Data Columns: 33
(_dystack pid=4124654) Label Column:       

✅ 훈련 완료! (소요시간: 131.2분)

📈 포괄적 성능 평가
🎯 핵심 성능 지표:
  정확도: 0.9949
  AUC 점수: 0.9999
  훈련 시간: 131.2분
  예측 시간: 75.82초
  예측 속도: 7064 샘플/초

📊 상세 분류 리포트:
              precision    recall  f1-score   support

          정상       0.99      1.00      0.99    208982
          이상       1.00      0.99      1.00    326644

    accuracy                           0.99    535626
   macro avg       0.99      0.99      0.99    535626
weighted avg       0.99      0.99      0.99    535626


🔍 혼동 행렬:
실제\예측    정상    이상
정상        208003    979
이상        1757   324887

🏆 모델 리더보드 (상위 5개):
  WeightedEnsemble_L4: 0.9999
  WeightedEnsemble_L3: 0.9999
  LightGBM_BAG_L3: 0.9999
  LightGBM_BAG_L2: 0.9999
  LightGBM_BAG_L1: 0.9999

⚖️ 규칙 기반 vs ML 비교:
  규칙 기반 정확도: 0.7242
  ML 모델 정확도: 0.9949
  개선율: +37.4%

⚠️ 특성 중요도 분석 불가

🎉 훈련 파이프라인 완료!
🎯 최종 AUC: 0.9999
🎯 최종 정확도: 0.9949
⏱️ 총 훈련 시간: 131.2분

✅ 훈련 성공!
💾 모델 저장 경로: ./enhanced_ship_anomaly_models_b2
💾 규칙 가중치 저장: ./rule_weights.json

🔮 STEP 2: 테스트 데이터 예측
-----------------------

In [ ]:
import pandas as pd
import numpy as np
import torch
import os
import json
import warnings
warnings.filterwarnings('ignore')

from autogluon.tabular import TabularPredictor

class ShipAnomalyPredictor:
    """선박 이상행동 탐지 시스템 - 예측 버전"""
    
    def __init__(self):
        self.rule_weights = {}
        self.rule_stats = {}
        
    def load_rule_weights(self, weights_path='./rule_weights.json'):
        """저장된 규칙 가중치를 로드"""
        if os.path.exists(weights_path):
            with open(weights_path, 'r') as f:
                self.rule_weights = json.load(f)
            print(f"📂 규칙 가중치 로드: {weights_path}")
            print("🔧 로드된 가중치:")
            for rule, weight in self.rule_weights.items():
                print(f"  {rule}: {weight:.3f}")
            
            # 통계 정보도 로드 (있다면)
            stats_path = weights_path.replace('.json', '_stats.json')
            if os.path.exists(stats_path):
                with open(stats_path, 'r') as f:
                    self.rule_stats = json.load(f)
                print(f"📊 규칙 통계도 로드됨")
            
            return True
        else:
            print(f"⚠️ 가중치 파일을 찾을 수 없습니다: {weights_path}")
            print("기본 가중치를 사용합니다")
            return False

    def create_rule_based_features(self, df, weights_path='./rule_weights.json'):
        """6가지 규칙 기반 특성 생성 (테스트용)"""
        print("🔧 도메인 지식 기반 특성 엔지니어링...")
        
        df_enhanced = df.copy()
        
        # 규칙 1: 외국국적 + 저속항해 + 비정박지
        df_enhanced['rule1_foreign_slow_non_anchor'] = (
            (df['is_korean_ship'] == 0) & 
            (df['anchor_zone_ratio'] < 0.8) & 
            ((df['low_speed_ratio'] > 0.7) | (df['avg_sog'] <= 5))
        ).astype(int)
        
        # 규칙 2: AIS OFF 이상 + 비정박지
        df_enhanced['rule2_ais_off_non_anchor'] = (
            (df['anchor_zone_ratio'] < 0.8) & 
            (df['off_events'] >= 3)
        ).astype(int)
        
        # 규칙 3: 특수해역 다중진입
        df_enhanced['rule3_special_zone_frequent'] = (
            df['num_zone_entries'] > 50
        ).astype(int)
        
        # 규칙 4: 급변침 + 비정박지
        df_enhanced['rule4_sharp_turn_non_anchor'] = (
            (df['anchor_zone_ratio'] < 0.8) & 
            (df['sharp_turns'] >= 1)
        ).astype(int)
        
        # 규칙 5: 저속 + 왕복항해
        df_enhanced['rule5_slow_back_forth'] = (
            (df['avg_sog'] <= 5) & 
            (df['back_forth_count'] >= 2)
        ).astype(int)
        
        # 규칙 6: 금지구역 진입
        df_enhanced['rule6_restricted_zone'] = df['restricted_zone_flag']
        
        # 저장된 가중치 로드
        loaded = self.load_rule_weights(weights_path)
        if loaded:
            rule_weights = self.rule_weights
        else:
            # 로드 실패 시 기본값 사용
            rule_weights = {
                'rule6_restricted_zone': 1.0,
                'rule1_foreign_slow_non_anchor': 0.977,  # 실제 데이터 기반 업데이트된 값
                'rule3_special_zone_frequent': 0.964,   # 실제 데이터 기반 업데이트된 값
                'rule4_sharp_turn_non_anchor': 0.752,   # 실제 데이터 기반 업데이트된 값
                'rule2_ais_off_non_anchor': 0.0,
                'rule5_slow_back_forth': 0.0
            }
            self.rule_weights = rule_weights
        
        # 종합 규칙 점수 (실제 데이터 기반 가중합)
        df_enhanced['rule_composite_score'] = 0
        for rule, weight in rule_weights.items():
            if rule in df_enhanced.columns:
                df_enhanced['rule_composite_score'] += df_enhanced[rule] * weight
        
        # 규칙 기반 예측
        df_enhanced['rule_based_prediction'] = (
            (df_enhanced['rule1_foreign_slow_non_anchor'] == 1) |
            (df_enhanced['rule2_ais_off_non_anchor'] == 1) |
            (df_enhanced['rule3_special_zone_frequent'] == 1) |
            (df_enhanced['rule4_sharp_turn_non_anchor'] == 1) |
            (df_enhanced['rule5_slow_back_forth'] == 1) |
            (df_enhanced['rule6_restricted_zone'] == 1)
        ).astype(int)
        
        print(f"✅ 규칙 기반 특성 8개 생성 완료")
        
        return df_enhanced
    
    def hybrid_prediction(self, predictor, X_new):
        """하이브리드 예측 (규칙 + ML)"""
        # ML 예측
        ml_pred = predictor.predict(X_new)
        ml_proba = predictor.predict_proba(X_new)
        
        # 규칙 기반 예측
        if 'rule_based_prediction' in X_new.columns:
            rule_pred = X_new['rule_based_prediction'].values
            
            # 하이브리드 예측: 규칙이 이상이면 이상, 아니면 ML 예측 사용
            hybrid_pred = np.maximum(rule_pred, ml_pred)
            
            return hybrid_pred, ml_proba
        else:
            return ml_pred, ml_proba

def predict_test_data(test_file_path, model_path='./enhanced_ship_anomaly_models_b2', output_path='./test.csv'):
    """학습된 모델로 test 데이터 예측"""
    
    print("🔮 Test 데이터 예측 시작")
    print("=" * 50)
    
    # 예측기 인스턴스 생성
    predictor_system = ShipAnomalyPredictor()
    
    try:
        # 학습된 모델 로드
        print(f"📂 모델 로딩: {model_path}")
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"모델 경로가 존재하지 않습니다: {model_path}")
        
        predictor = TabularPredictor.load(model_path)
        print("✅ 모델 로딩 완료")
        
        # 테스트 데이터 로드
        print(f"📁 테스트 데이터 로딩: {test_file_path}")
        test_df = pd.read_csv(test_file_path)
        print(f"✅ 테스트 데이터 로딩 완료: {test_df.shape}")
        
        # MMSI 보존
        mmsi_column = test_df['MMSI'].copy() if 'MMSI' in test_df.columns else None
        
        # 동일한 특성 엔지니어링 적용 (테스트 모드)
        print("⚙️ 특성 엔지니어링 적용...")
        test_final = predictor_system.create_rule_based_features(test_df)
        
        # 예측용 특성 준비
        feature_columns = [col for col in test_final.columns if col not in ['result', 'MMSI']]
        X_test = test_final[feature_columns]
        
        print(f"🔧 예측용 특성: {len(feature_columns)}개")
        print(f"📊 예측할 데이터: {X_test.shape[0]:,}개")
        
        # 하이브리드 예측 수행
        print("🔮 예측 수행...")
        start_time = pd.Timestamp.now()
        
        hybrid_pred, ml_proba = predictor_system.hybrid_prediction(predictor, X_test)
        
        end_time = pd.Timestamp.now()
        prediction_time = (end_time - start_time).total_seconds()
        
        print(f"✅ 예측 완료! (소요시간: {prediction_time:.2f}초)")
        print(f"🚀 예측 속도: {len(X_test)/prediction_time:.0f} 샘플/초")
        
        # 결과를 TRUE/FALSE로 변환
        result_labels = ['False' if pred == 0 else 'True' for pred in hybrid_pred]
        
        # 결과 DataFrame 생성
        result_df = pd.DataFrame()
        
        if mmsi_column is not None:
            result_df['MMSI'] = mmsi_column
        
        result_df['result'] = result_labels
        
        # 예측 결과 통계
        pred_counts = pd.Series(result_labels).value_counts()
        print(f"\n📊 예측 결과 분포:")
        for label, count in pred_counts.items():
            percentage = count / len(result_labels) * 100
            print(f"  {label}: {count:,}개 ({percentage:.1f}%)")
        
        # 규칙 기반 vs 하이브리드 비교
        if 'rule_based_prediction' in X_test.columns:
            rule_anomalies = X_test['rule_based_prediction'].sum()
            hybrid_anomalies = sum(hybrid_pred)
            print(f"\n⚖️ 규칙 vs 하이브리드:")
            print(f"  규칙 기반 이상: {rule_anomalies:,}개")
            print(f"  하이브리드 이상: {hybrid_anomalies:,}개")
            print(f"  ML 추가 탐지: {hybrid_anomalies - rule_anomalies:,}개")
        
        # 결과 저장
        result_df.to_csv(output_path, index=False)
        print(f"\n💾 결과 저장 완료: {output_path}")
        
        # 샘플 결과 출력
        print(f"\n📋 예측 결과 샘플 (상위 10개):")
        print(result_df.head(10).to_string(index=False))
        
        # 규칙별 상세 분석
        print(f"\n🔍 규칙별 탐지 현황:")
        for rule_name in ['rule1_foreign_slow_non_anchor', 'rule3_special_zone_frequent', 
                         'rule4_sharp_turn_non_anchor', 'rule6_restricted_zone']:
            if rule_name in X_test.columns:
                count = X_test[rule_name].sum()
                print(f"  {rule_name}: {count}개")
        
        print(f"\n🎉 예측 완료!")
        print(f"📁 결과 파일: {output_path}")
        print("=" * 50)
        
        return result_df
        
    except Exception as e:
        print(f"❌ 예측 중 오류 발생: {e}")
        return None

# 사용 예시
if __name__ == "__main__":
    print("🚀 선박 이상행동 탐지 시스템 - 테스트 예측")
    print("=" * 60)
    
    # 필수 파일 존재 확인
    model_path = './enhanced_ship_anomaly_models_b2'
    weights_path = './rule_weights.json'
    
    print("📋 필수 파일 확인:")
    print(f"  모델 폴더: {'✅' if os.path.exists(model_path) else '❌'} {model_path}")
    print(f"  가중치 파일: {'✅' if os.path.exists(weights_path) else '❌'} {weights_path}")
    
    if not os.path.exists(model_path):
        print("\n❌ 오류: 훈련된 모델이 없습니다!")
        print("먼저 train.py를 실행하여 모델을 훈련하세요.")
        exit(1)
    
    # 테스트 파일 경로 설정
    test_file_path = './20240701.csv'  # 실제 테스트 데이터 경로로 변경
    output_path = './test_predictions_final.csv'  # 결과 저장 경로
    
    print(f"\n📁 테스트 데이터: {test_file_path}")
    print(f"💾 결과 저장 경로: {output_path}")
    print("-" * 60)
    
    # 테스트 데이터 예측 실행
    predictions = predict_test_data(
        test_file_path=test_file_path,
        model_path=model_path,
        output_path=output_path
    )
    
    if predictions is not None:
        print(f"\n🎉 전체 예측 완료!")
        print(f"📊 총 {len(predictions):,}개 선박 예측 완료")
        print(f"💾 최종 결과: {output_path}")
        
        # 요약 통계
        anomaly_count = (predictions['result'] == 'True').sum()
        total_count = len(predictions)
        anomaly_rate = anomaly_count / total_count * 100
        
        print(f"\n📈 최종 요약:")
        print(f"  전체 선박: {total_count:,}개")
        print(f"  이상 선박: {anomaly_count:,}개 ({anomaly_rate:.1f}%)")
        print(f"  정상 선박: {total_count - anomaly_count:,}개 ({100-anomaly_rate:.1f}%)")
        
    else:
        print("❌ 예측 실패")
        print("⚠️ 테스트 데이터 경로와 설정을 확인해주세요")